# ETL Pipeline

This notebook cleans the raw data and writes an hourly aggregated `masterdata_<YEAR>.csv` where `<YEAR>` corresponds to the data folder.

In [ ]:
import pandas as pd
import glob, os, re
from dotenv import load_dotenv
load_dotenv()

PATH_DATA = os.getenv('path_to_data')
PATH_MASTER = os.getenv('path_to_master')
match = re.search(r'(\d{4})', PATH_DATA or '')
YEAR = os.getenv('YEAR', match.group(1) if match else '2024')

## Weather data

In [ ]:
def fix_dates(df):
    df['DATUM'] = df['MESS_DATUM'].astype(str)
    df['DATUM'] = pd.to_datetime(df['DATUM'].str.slice(0, 10), format='%Y%m%d%H')
    df.set_index('DATUM', inplace=True)
    return df

def rename_columns(df):
    renames = {}  # add column mappings if necessary
    return df.rename(columns=renames)

def remove_duplicate_columns(df):
    return df.drop(columns=['eor', 'STATIONS_ID', 'MESS_DATUM'], errors='ignore')

def clean_weather_files():
    for file in glob.glob(os.path.join(PATH_DATA, 'schnarrenberg_dwd*.csv')):
        df = pd.read_csv(file, delimiter=';')
        df = fix_dates(df)
        df = rename_columns(df)
        out = os.path.join(PATH_DATA, f'clean_{os.path.basename(file)}')
        df.to_csv(out)

def aggregate_weather():
    files = glob.glob(os.path.join(PATH_DATA, 'clean_schnarrenberg_dwd*.csv'))
    start = f'{YEAR}-01-01'
    end = f'{YEAR}-12-31 23:00:00'
    date_range = pd.date_range(start, end, freq='h')
    df = pd.DataFrame(index=date_range)
    for file in files:
        df_new = pd.read_csv(file)
        df_new = remove_duplicate_columns(df_new)
        df_new['DATUM'] = pd.to_datetime(df_new['DATUM'])
        df_new.set_index('DATUM', inplace=True)
        df = df.merge(df_new, left_index=True, right_index=True, how='left', suffixes=('', '_y'))
    df.dropna(how='all', inplace=True)
    df.to_csv(os.path.join(PATH_DATA, 'clean_wetter_komplett.csv'))
    return df

## Measurement data

In [ ]:
def clean_measurements():
    df = pd.read_csv(os.path.join(PATH_MASTER, f'messungen_{YEAR}.csv'), delimiter=';').T
    df.columns = df.iloc[0]
    df = df.iloc[1:].copy()
    df.columns = ['zeit','ecoli','entro','pos_neg']
    df.index.rename('DATUM', inplace=True)
    for col in ['ecoli','entro']:
        df[col] = df[col].str.replace('>800','8000').str.replace('>80','8000').str.replace('>','').astype(float)
    df.reset_index(inplace=True)
    month_map={'Jan':'01.','Febr':'02.','Mär':'03.','Apr':'04.','Mai':'05.','Jun':'06.','Jul':'07.','Aug':'08.','Sep':'09.','Okt':'10.','Nov':'11.','Dez':'12.'}
    for k,v in month_map.items():
        df['DATUM'] = df['DATUM'].str.replace(k,v)
    def fill_year(d):
        return d if d.endswith(str(YEAR)[-2:]) else d + ' ' + str(YEAR)[-2:]
    def fix_date(d):
        parts=d.split(' ')
        return '20'+parts[-1]+'-'+parts[1][:-1]+'-'+parts[0][:-1]
    def add_time(date,time):
        hour=time.str.split(':').str[0]
        return date+' '+hour+':00:00'
    df['DATUM']=df['DATUM'].apply(fill_year).apply(fix_date)
    df['DATUM']=add_time(df['DATUM'],df['zeit'])
    df['DATUM']=pd.to_datetime(df['DATUM']).dt.round('h')
    df.set_index('DATUM', inplace=True)
    df.to_csv(os.path.join(PATH_MASTER, f'messungen_clean_{YEAR}.csv'))
    return df

## Additional water temperature

In [ ]:
def clean_hofen_temp(master_df):
    df = pd.read_csv(os.path.join(PATH_DATA, 'hofen_wassertemp.csv'), delimiter=';')
    df['Datum'] = pd.to_datetime(df['Datum'], format='%d.%m.%Y')
    df = df[df['Datum'] > pd.to_datetime(f'{YEAR}-01-01')]
    df = df[['Datum', 'Tagesmittelwert']].set_index('Datum')
    df['Tagesmittelwert'] = pd.to_numeric(df['Tagesmittelwert'], errors='coerce')
    df.rename(columns={'Tagesmittelwert': 'Wassertemperatur Hofen'}, inplace=True)
    df = df.resample('h').bfill().ffill()
    return master_df.merge(df, left_index=True, right_index=True, how='left')

## Combine all data

In [ ]:
def create_masterdata():
    weather = aggregate_weather()
    messungen = clean_measurements()
    df = weather.merge(messungen, left_index=True, right_index=True, how='left')
    df_wendlingen = pd.read_csv(os.path.join(PATH_DATA, 'wendlingen_messwerte.csv'), delimiter=';', parse_dates=True, index_col=0)
    df_wendlingen = df_wendlingen.pivot(index='Datum', columns='Parameter', values='Tagesmittelwert')
    df_wendlingen.index = pd.to_datetime(df_wendlingen.index, format='%d.%m.%Y')
    df_wendlingen = df_wendlingen.resample('1h').first().ffill()
    for col in df_wendlingen.columns:
        df_wendlingen[col] = pd.to_numeric(df_wendlingen[col].str.replace(',', '.'), errors='coerce')
    df = df.merge(df_wendlingen, left_index=True, right_index=True, how='left')
    df = clean_hofen_temp(df)
    df = df.loc[f'{YEAR}']
    df.index.name = 'zeit'
    df.to_csv(os.path.join(PATH_MASTER, f'masterdata_{YEAR}.csv'))
    return df

In [ ]:
# Run the ETL pipeline
create_masterdata()